In [1]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()

from langchain import PromptTemplate
from langchain.llms import GPT4All, OpenAI
from langchain.callbacks.streaming_stdout_final_only import \
    FinalStreamingStdOutCallbackHandler
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain


In [63]:
def ReviewGeneration(product_name, input_criteria_score : dict) -> None:
    """
    This function expects a dictionary of the form:
    input_criteria_score = [{"criteria_0":score_0}, {"criteria_1":score_1}, ...., {"criteria_n":score_n}]
    if not, this function returns None.
    """
    # Global Variables
    N = len(input_criteria_score[0]) #number of criteria

    # transforming score into adjectives
    criteria = [{"0":"Very bad", "1":"Bad", "2":"Not so good", "3":"Acceptable", "4":"Good", "5":"Very good"}]

    # Decompressing input_criteria_score dictionary
    prefix = []
    input_criteria = []
    input_adjectives = []
    for key, value in input_criteria_score[0].items():
        input_criteria.append(key)
        input_adjectives.append(criteria[0][str(input_criteria_score[0][key])])
        pref = f"{key}:{criteria[0][str(input_criteria_score[0][key])]}"
        prefix.append(pref)

    # Criteria, Adjective and Feeling lists
    criteria_list, adjective_list, feeling_list = [f"criteria_{n}" for n in range(N)],\
                                                  [f"adjective_{n}" for n in range(N)],\
                                                  [f"feeling_{n}" for n in range(N)]

       
    # template for PromptTemplate()
    def template(prefix):
        """
            This function build a template with a set of instructions to be passed to the model.
        """
        temp = f" You are a costumer. I want you to generate a product review on {product_name}\
               considering the following criteria: {prefix}. Be precise and concise."

        return temp
    
    template = template(prefix)

    #input variables for PromptTemplate()
    def input_variable(product_name, criteria_list,  adjective_list,  feeling_list=None,  **kwargs):
        """
            This function builds the input_variables necessary for the PromptTemplate()
            Here, we have made feeling_list optional. input_variable() can be called without it!
        """
        if feeling_list is not None:
            input_var = [f"{product_name}"] + criteria_list + adjective_list + feeling_list
        else:
            input_var = [f"{product_name}"] + criteria_list + adjective_list

        return input_var

    #instantiating input_variable()
    input_variables = input_variable(product_name, criteria_list,  adjective_list)

    #instantiating the model
    llm = OpenAI(temperature=0.9)

    # building the prompt
    prompt = PromptTemplate(
        input_variables = input_variables,
        template = template
            )
    def Prompt(**kwargs):
        if N == 1:
            response = llm(prompt.format(criteria_0=input_criteria[0], adjective_0=input_adjectives[0])                  
                          )
        if N == 2:
            response = llm(prompt.format(criteria_0=input_criteria[0], criteria_1=input_criteria[1],
                          adjective_0=input_adjectives[0], adjective_1=input_adjectives[1])                          
                          )
        if N == 3:
            response = llm(prompt.format(criteria_0=input_criteria[0], criteria_1=input_criteria[1], criteria_2=input_criteria[2],\
                          adjective_0=input_adjectives[0], adjective_1=input_adjectives[1], adjective_2=input_adjectives[2])                          
                          )
        if N == 4:
            response = llm(prompt.format(criteria_0=input_criteria[0], criteria_1=input_criteria[1], criteria_2=input_criteria[2], criteria_3=input_criteria[3],\
                          adjective_0=input_adjectives[0], adjective_1=input_adjectives[1], adjective_2=input_adjectives[2], adjective_3=input_adjectives[3])                          
                          )
        if N == 5:
            response = llm(prompt.format(criteria_0=input_criteria[0], criteria_1=input_criteria[1], criteria_2=input_criteria[2], criteria_3=input_criteria[3], criteria_4=input_criteria[4],\
                          adjective_0=input_adjectives[0], adjective_1=input_adjectives[1], adjective_2=input_adjectives[2], adjective_3=input_adjectives[3], adjective_4=input_adjectives[4])                          
                          )
        
        return response

    # review_generation = Prompt()
    return 1 #review_generation

In [50]:
#example
product_name = "iphone"

#fake values
score = [0, 4, 2, 3, 1]
criteria_score = [{"Design and Build Quality":score[0], "Display":score[1], "Performance":score[2], "Camera Quality":score[3], "Battery Life":score[4]}]

In [51]:
ReviewGeneration(product_name, criteria_score)

"\n\nI recently purchased the iPhone and I must say, I am quite disappointed with the design and build quality. It feels flimsy and cheap, definitely not worth the price I paid for it. However, the display is good and the colors are vibrant, so that's a plus.\n\nWhen it comes to performance, I have to say it's not so good. Apps often freeze and it takes forever to load anything. This can be quite frustrating, especially when I need to get things done quickly.\n\nThe camera quality is acceptable, but nothing to write home about. It takes decent pictures in good lighting, but the photos tend to be grainy in low light situations. As for battery life, it's bad. I find myself constantly needing to charge my phone throughout the day, which can be a hassle.\n\nIn conclusion, while the iPhone may have a good display and acceptable camera quality, the overall design and build quality, performance, and battery life do not meet my expectations. I am disappointed with my purchase and would not rec

In [61]:
#example
product_name = "jean"

score = [4, 3, 2, 3, 0]
criteria_score = [{
        "Design": score[0],
        "Comfort": score[1],
        "Durability": score[2],
        "Fit": score[3],
        "Price": score[4]
    }]

In [57]:
ReviewGeneration(product_name, criteria_score)

'\nI recently purchased a pair of jeans and after wearing them for a while, I wanted to share my experience with others. Overall, I found that the jeans had a good design, acceptable comfort, and an acceptable fit. However, I was disappointed with the durability and the price of the jeans.\n\nStarting with the positives, I really liked the design of the jeans. The color and style were exactly what I was looking for and I received many compliments when wearing them. The comfort level was also acceptable, though not exceptional. The fabric was soft and had some stretch, but after a few hours of wear, I did find myself wanting to take them off.\n\nIn terms of fit, I found the jeans to be acceptable. They fit well in the waist and hips, but I did notice some bagginess in the thighs. However, overall, I was happy with how they looked on me.\n\nUnfortunately, the durability of the jeans was not up to my expectations. After only a few wears, I started to notice some fraying in the seams and t

In [64]:
ReviewGeneration(product_name, criteria_score)

1

In [27]:
!pwd

/home/kenmoe/code/jo25425/review-assistant/notebooks


In [55]:
GPT4ALL_MODELS_URL = "https://gpt4all.io/models/gguf/"

def download_gpt4all_model(model_name: str, model_path: str):
    url = GPT4ALL_MODELS_URL + model_name
    response = requests.get(url, stream=True)

    with open(model_path, mode='wb') as out:
        for chunk in response.iter_content(chunk_size=10 * 1024):
            out.write(chunk)

def load_model_review(mode: str, model_name: str):

    # OpenAI
    if mode == 'openai':
        llm = OpenAI(model_name=model_name, openai_api_key=OPENAI_API_KEY, temperature=0.9)
        return llm

    # Local
    if mode == 'local':
        model_name = "mistral-7b-openorca.gguf2.Q4_0.gguf"
        MODEL_DIR = "./models"
        #model_path = os.path.join(MODEL_DIR, model_name)
    
        model_path = os.path.join(MODEL_DIR, model_name)

        if not os.path.exists(model_path) \
            and os.path.isfile(model_path):
            download_gpt4all_model(model_name, model_path)

        callbacks = [FinalStreamingStdOutCallbackHandler()]
        llm = GPT4All(model=model_path, callbacks=callbacks, verbose=True)
        return llm

In [52]:
       
def ReviewGeneration(product_name, input_criteria_score : dict) -> None:
    """
    This function expects a dictionary of the form:
    input_criteria_score = [{"criteria_0":score_0}, {"criteria_1":score_1}, ...., {"criteria_n":score_n}]
    if not, this function returns None.
    """
    # Global Variables
    N = len(input_criteria_score[0]) #number of criteria

    # transforming score into adjectives
    criteria = [{"0":"Very bad", "1":"Bad", "2":"Not so good", "3":"Acceptable", "4":"Good", "5":"Very good"}]

    # Decompressing input_criteria_score dictionary
    prefix = []
    input_criteria = []
    input_adjectives = []
    for key, value in input_criteria_score[0].items():
        input_criteria.append(key)
        input_adjectives.append(criteria[0][str(input_criteria_score[0][key])])
        pref = f"{key}:{criteria[0][str(input_criteria_score[0][key])]}"
        prefix.append(pref)

    # Criteria, Adjective and Feeling lists
    criteria_list, adjective_list, feeling_list = [f"criteria_{n}" for n in range(N)],\
                                                  [f"adjective_{n}" for n in range(N)],\
                                                  [f"feeling_{n}" for n in range(N)]

       
    # template for PromptTemplate()
    def template(prefix):
        """
            This function builds a template with a set of instructions to be passed to the model.
        """
        temp = f" You are a costumer. I want you to generate a product review on {product_name}\
               considering the following criteria: {prefix}. Be precise and concise."

        return temp

    #instantiating the model
    model_name = "mistral-7b-openorca.gguf2.Q4_0.gguf"
    mode = "local"
    #MODEL_DIR = "./models"
    #model_path = os.path.join(MODEL_DIR, model_name)
    
    template = template(prefix)

    prompt = PromptTemplate.from_template(template)

    llm = load_model_review(mode, model_name)

    llm_chain = LLMChain(prompt=prompt, llm=llm)

    question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

    review_generation = llm_chain.run(question)
    
    return review_generation

In [53]:
#example
product_name = "jean"

score = [4, 3, 2, 3, 0]
criteria_score = [{
        "Design": score[0],
        "Comfort": score[1],
        "Durability": score[2],
        "Fit": score[3],
        "Price": score[4]
    }]

In [54]:
ReviewGeneration(product_name, criteria_score)

ValidationError: 1 validation error for GPT4All
__root__
  cannot inherit from both a TypedDict type and a non-TypedDict base class (type=type_error)

In [67]:
https://python.langchain.com/docs/integrations/llms/openai
https://python.langchain.com/docs/integrations/llms/gpt4all

/home/kenmoe/code/jo25425/review-assistant/notebooks


In [8]:
model_path = "./models/mistral-7b-openorca.gguf2.Q4_0.gguf"

In [9]:

callbacks = [StreamingStdOutCallbackHandler()]
llm = GPT4All(model=model_path, callbacks=callbacks, verbose=True)

ValidationError: 1 validation error for GPT4All
__root__
  cannot inherit from both a TypedDict type and a non-TypedDict base class (type=type_error)

In [5]:
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

NameError: name 'local_path' is not defined

In [62]:

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

llm_chain.run(question)

NameError: name 'LLMChain' is not defined